## Cell 1: Environment Setup

In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import importlib

# Import and reload scanner module (ensures latest code changes are used)
import sp500_scanner
importlib.reload(sp500_scanner)
from sp500_scanner import get_sp500_tickers, scan_sp500, filter_buy_signals, create_excel_output, create_pdf_report, cleanup_old_scans

print("✓ Imports complete")
print(f"Ready to scan S&P 500 as of {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✓ Imports complete
Ready to scan S&P 500 as of 2026-01-05 22:05


## Cell 2: Run Full S&P 500 Scan

**This will take 5-10 minutes to complete.**

Parameters:
- `daily_bars=60` - 60 days of daily data
- `weekly_bars=52` - 52 weeks of weekly data  
- `concurrency=15` - 15 parallel threads (adjust if needed)

In [2]:
# Run the scan
start_time = datetime.now()
print(f"🔍 Starting S&P 500 scan at {start_time.strftime('%H:%M:%S')}...\n")

results_df = scan_sp500(daily_bars=60, weekly_bars=52, concurrency=15)

elapsed = (datetime.now() - start_time).total_seconds()
print(f"\n✓ Scan completed in {elapsed:.1f} seconds")

🔍 Starting S&P 500 scan at 22:05:30...

Fetching S&P 500 ticker list...
✓ Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

🔍 Scanning 503 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 15 threads
✓ [4/503] ALB    -> FULL HOLD + ADD      $146.11
✓ [7/503] ABBV   -> FULL HOLD + ADD      $220.18
✓ [14/503] AFL    -> FULL HOLD + ADD      $112.12
✓ [18/503] GOOG   -> FULL HOLD + ADD      $317.32
✓ [19/503] ALL    -> FULL HOLD + ADD      $205.64
✓ [24/503] GOOGL  -> FULL HOLD + ADD      $316.54
✓ [25/503] AXP    -> FULL HOLD + ADD      $379.83
✓ [26/503] AIG    -> FULL HOLD + ADD      $84.37
✓ [28/503] AMZN   -> FULL HOLD + ADD      $233.06
✓ [32/503] ADI    -> FULL HOLD + ADD      $277.29
✓ [33/503] AME    -> FULL HOLD + ADD      $211.38
✓ [34/503] AMGN   -> FULL HOLD + ADD      $320.72
✓ [35/503] APH    -> FULL HOLD + ADD      $139.89
✓ [36/503] APO    -> FULL HOLD + ADD      $149.76
✓ [37/503] AAPL   -> FULL HOLD + ADD      $267.26
✓ [40/503] AMAT 

## Cell 3: Filter and Display Results by Confluence

In [3]:
# Filter for FULL HOLD + ADD signals
buy_df = filter_buy_signals(results_df, 'FULL HOLD + ADD')

print(f"{'='*80}")
print(f"🎯 FULL HOLD + ADD SIGNALS: {len(buy_df)} stocks")
print(f"{'='*80}\n")

# Breakdown by confluence
print("📊 Breakdown by Confluence:")
confluence_counts = buy_df['confluence'].value_counts()
for conf, count in confluence_counts.items():
    print(f"  {conf:12s}: {count:3d} stocks")

# Save to workspace scanner_results folder
results_dir = Path.cwd() / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
excel_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
pdf_path = results_dir / f'scanner_report_{timestamp}.pdf'

# Generate Excel
create_excel_output(buy_df, excel_path)
print(f"\n✓ Excel saved to: {excel_path}")

# Generate PDF
print(f"\n📄 Creating PDF research document...")
create_pdf_report(buy_df, results_df, pdf_path, timestamp)
print(f"✓ PDF saved to: {pdf_path}")

# Cleanup old scans (keep 7 most recent, archive rest)
print("\n📁 Managing scan history...")
cleanup_old_scans(results_dir, max_files=7)

🎯 FULL HOLD + ADD SIGNALS: 201 stocks

📊 Breakdown by Confluence:
  WEAK        : 129 stocks
  EXTENDED    :  49 stocks
  BALANCED    :  23 stocks
✓ Excel file created: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2207.xlsx
  - All: 201 stocks
  - Balanced: 23 stocks
  - Extended: 49 stocks
  - Weak: 129 stocks

✓ Excel saved to: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2207.xlsx

📄 Creating PDF research document...
✓ PDF report created: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2207.pdf
  - 23 BALANCED stocks detailed
  - 49 EXTENDED stocks in watch list
✓ PDF saved to: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2207.pdf

📁 Managing scan history...


## Cell 4: Display BALANCED Stocks (Priority Buy List)

In [4]:
# Filter for BALANCED confluence only
balanced_df = buy_df[buy_df['confluence'] == 'BALANCED'].copy()
balanced_df = balanced_df.sort_values('ticker')

print(f"🎯 BALANCED STOCKS (Priority Buy Candidates): {len(balanced_df)}\n")

if not balanced_df.empty:
    # Display key columns
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in balanced_df.columns]
    
    print(balanced_df[available_cols].to_string(index=False))
    
    print(f"\n✅ These {len(balanced_df)} stocks have:")
    print("  ✓ FULL HOLD + ADD signal (Weekly P1 + Daily P1)")
    print("  ✓ BALANCED confluence (healthy technical setup)")
    print("  ✓ 'Enter on Dip' recommendation")
else:
    print("No BALANCED stocks found in this scan.")

🎯 BALANCED STOCKS (Priority Buy Candidates): 23

ticker  current_price recommendation        d50       d200        w10       w200
  AAPL        267.260   Enter on Dip 272.940846 231.964810 273.594434 191.501617
  AMGN        320.720   Enter on Dip 323.211371 294.279543 326.479382 261.091446
 BRK-B        498.565   Enter on Dip 498.280900 497.781074 499.692999 394.245849
  CBOE        251.830   Enter on Dip 251.710200 236.585174 252.869002 171.415324
   CEG        354.940   Enter on Dip 360.465316 313.334904 356.867160 172.377177
  CINF        161.960   Enter on Dip 162.195634 150.708209 163.078886 118.656421
  CSCO         75.580   Enter on Dip  75.461230  67.038620  75.954564  51.337482
    EW         84.180   Enter on Dip  84.378400  78.079500  85.105000  81.503850
   EXE        106.820   Enter on Dip 112.483108 106.300980 113.200696  85.873114
  GILD        118.300   Enter on Dip 122.147535 112.444060 122.189639  79.935261
   HSY        179.920   Enter on Dip 179.916782 174.241398 1

## Cell 5: Display EXTENDED Stocks (Watch List)

In [5]:
# Filter for EXTENDED confluence
extended_df = buy_df[buy_df['confluence'] == 'EXTENDED'].copy()
extended_df = extended_df.sort_values('ticker')

print(f"⏳ EXTENDED STOCKS (Wait for Pullback): {len(extended_df)}\n")

if not extended_df.empty:
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200']
    available_cols = [col for col in display_cols if col in extended_df.columns]
    
    # Show first 20
    print(extended_df[available_cols].head(20).to_string(index=False))
    
    if len(extended_df) > 20:
        print(f"\n... and {len(extended_df) - 20} more (see Excel file)")
    
    print(f"\n⚠️ These stocks have strong signals but are technically overbought.")
    print("  → Wait for pullback to support before entering")
else:
    print("No EXTENDED stocks found.")

⏳ EXTENDED STOCKS (Wait for Pullback): 49

ticker  current_price   recommendation        d50       d200
  ACGL         94.440 Wait for Support  92.024800  91.303800
   AFL        112.119 Wait for Support 109.937330 106.050442
   AIZ        240.371 Wait for Support 226.027442 206.790098
  AMZN        233.060 Wait for Support 231.828401 217.216100
   APH        139.890 Wait for Support 136.634649 107.054605
   AXP        379.835 Wait for Support 365.784966 316.681215
   CAH        205.470 Wait for Support 198.556930 162.191908
    CB        315.220 Wait for Support 297.326000 285.489231
   CCL         31.505 Wait for Support  27.741500  26.361275
   CME        275.060 Wait for Support 272.477866 269.333022
   CRH        128.325 Wait for Support 119.775500 105.144388
  CTSH         81.630 Wait for Support  77.400219  74.244442
   CVS         80.420 Wait for Support  78.769200  70.199836
    DG        138.930 Wait for Support 115.717600 105.667301
  EXPD        154.330 Wait for Support 141

## Cell 6: Compare with Current Holdings (stocks.txt)

In [6]:
from technical_analysis import parse_stocks_file

# Load current holdings
stocks_file = Path.cwd() / 'stocks.txt'
individual_tickers, baskets = parse_stocks_file(stocks_file)

# Find BALANCED stocks NOT in your portfolio
balanced_tickers = set(balanced_df['ticker'].tolist())
current_tickers = set(individual_tickers)
new_opportunities = balanced_tickers - current_tickers

print(f"📋 Current Portfolio: {len(current_tickers)} stocks")
print(f"🎯 BALANCED Opportunities: {len(balanced_tickers)} stocks")
print(f"\n✨ NEW BALANCED Stocks (not in your portfolio): {len(new_opportunities)}\n")

if new_opportunities:
    new_df = balanced_df[balanced_df['ticker'].isin(new_opportunities)].sort_values('ticker')
    display_cols = ['ticker', 'current_price', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in new_df.columns]
    print(new_df[available_cols].to_string(index=False))
    print(f"\n💡 Consider adding these {len(new_opportunities)} stocks to stocks.txt for detailed analysis")
else:
    print("All BALANCED opportunities are already in your portfolio!")

# Check which current holdings are also BALANCED
holdings_balanced = balanced_tickers & current_tickers
print(f"\n✅ Current holdings that are BALANCED: {len(holdings_balanced)}")
if holdings_balanced:
    print(f"   {', '.join(sorted(holdings_balanced))}")

📋 Current Portfolio: 13 stocks
🎯 BALANCED Opportunities: 23 stocks

✨ NEW BALANCED Stocks (not in your portfolio): 22

ticker  current_price        d50       d200        w10       w200
  AAPL        267.260 272.940846 231.964810 273.594434 191.501617
  AMGN        320.720 323.211371 294.279543 326.479382 261.091446
 BRK-B        498.565 498.280900 497.781074 499.692999 394.245849
  CBOE        251.830 251.710200 236.585174 252.869002 171.415324
   CEG        354.940 360.465316 313.334904 356.867160 172.377177
  CINF        161.960 162.195634 150.708209 163.078886 118.656421
  CSCO         75.580  75.461230  67.038620  75.954564  51.337482
    EW         84.180  84.378400  78.079500  85.105000  81.503850
   EXE        106.820 112.483108 106.300980 113.200696  85.873114
  GILD        118.300 122.147535 112.444060 122.189639  79.935261
   HSY        179.920 179.916782 174.241398 180.812688 191.171458
   IBM        294.960 303.772777 269.972437 303.520816 176.776666
  IDXX        682.230 6

## Cell 7: Overall Signal Distribution

In [7]:
print("📊 Complete S&P 500 Signal Distribution:\n")
print(results_df['signal'].value_counts().to_string())

print(f"\n\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"Total S&P 500 stocks analyzed: {len(results_df)}")
print(f"FULL HOLD + ADD signals: {len(buy_df)}")
print(f"  ├─ BALANCED (buy now): {len(buy_df[buy_df['confluence'] == 'BALANCED'])}")
print(f"  ├─ EXTENDED (watch): {len(buy_df[buy_df['confluence'] == 'EXTENDED'])}")
print(f"  └─ WEAK (skip): {len(buy_df[buy_df['confluence'] == 'WEAK'])}")
print(f"\nExcel file: {excel_path}")
print(f"PDF report: {pdf_path}")

📊 Complete S&P 500 Signal Distribution:

signal
FULL HOLD + ADD       201
HOLD MOST → REDUCE     79
FULL CASH / DEFEND     72
CASH                   52
REDUCE                 36
HOLD                   34
SCALE IN               21
LIGHT / CASH            7


Summary
Total S&P 500 stocks analyzed: 502
FULL HOLD + ADD signals: 201
  ├─ BALANCED (buy now): 23
  ├─ EXTENDED (watch): 49
  └─ WEAK (skip): 129

Excel file: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2207.xlsx
PDF report: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2207.pdf
